## Estimación de espectro de amplitud y fase de datos sísmicos 2D 

**Diciembre 2018**

Existe un problema intrínseco en la representación del contenido de energía y fase en el dominio de Fourier de datos sísmicos 3D. Lo que obtenemos, es una respuesta promedio del volumen sísmico (Fig. 1); mientras que de una traza sísmica (serie de tiempo) podemos numéricamente representar con gran precisión su contenido de energía y fase, no así para dos o más trazas sísmicas, de las que de manera legible solo podemos intepretar su respuesta promedio. 

Por curiosidad numérica calculo y represento los espectros de amplitud y fase de datos sísmicos bidimensionales. He elaborado este cuaderno de trabajo con la intención de ilustrar dos formar de estimar el espectro de amplitud de una sección sísmica con `Python`. <br> Los datos sísmicos son propiedad del Departamento de Energía de Nueva Escocia, distribuido por la compañia [`dGB Earth Sciences`](https://dgbes.com/), bajo una licencia CC BY-SA. Para este ejercicio, seleccioné la línea longitudinal no. 1153. 

<img src="Ejemplo-EspectroAmplitud.png" alt="drawing" style="float:center;width:600px;height:340px;border:left;"/>

<font size=2 color=black>**Figura 1.** Ejemplo típico de la representación del espectro de amplitud de un volumen de datos sísmicos. En la parte derecha inferior, se muestra el espectro de amplitud en decibeles. </font>

In [ ]:
import numpy as np
from scipy import signal
import matplotlib.pyplot as plt
from obspy.io.segy.segy import _read_segy

%matplotlib inline
plt.style.use('seaborn-paper')

**Sugerencia:** Sobre el estilo de gráficas, ver [aquí]([ver](https://tonysyu.github.io/raw_content/matplotlib-style-gallery/gallery.html)).

### Importar datos sísmicos en formato [SEGY](https://en.wikipedia.org/wiki/SEG-Y)

In [ ]:
xlinea_sismica = _read_segy('xline_1153.sgy', headonly=True)
seis = np.stack(t.data for t in xlinea_sismica.traces)

#### Parámetros de los datos sísmicos

In [ ]:
dt = 0.004
dim = seis.T.shape
dimz = seis.T.shape[0]
print('Tamaño de matriz =', dim)
print('Intervalo de muestreo =', dt, 'ms')
print('Longitud de grabación = ', dt * dimz, 's' )

#### Gráfica de los datos sísmicos

En la Fig. 2 se muestra la línea transversal de estudio.

In [ ]:
plt.figure(figsize=[22,9])

percen = np.percentile(seis, 99)

secc = plt.imshow(seis.T[:,:],vmin=-percen, vmax=percen, cmap="bwr",  aspect='auto',
                interpolation='gaussian')
cbar= plt.colorbar(secc, aspect=15, shrink=0.8)
cbar.set_label('Amplitud')
_ = plt.yticks(np.linspace(0,751,7), [0.0,0.5,1.0,1.5,2.0,2.5, 3.0])
_ = plt.xlabel('No. Traza', fontsize = 12, weight = 'semibold')
_ = plt.ylabel('Tiempo [s]', fontsize = 12, weight = 'semibold')
_ = plt.title('Línea transversal', fontsize=16, weight = 'semibold')
_ = plt.grid(True,alpha=0.6,linestyle=':')

<font size=2 color=black>**Figura 2.** Sección sísmica. </font>

### Naturaleza del problema: Espectros de Fourier de datos bidimensionales


Las Figuras 3 y 4 representan el espectro de amplitud en escala lineal y decibel de la sección sísmica de la Fig. 2, respectivamente; la Fig. 5 representa el espectro de fase. La legibilidad e interpretación de las gráficas anteriores no es sencilla debido a que se visualizan tres variables en una imagen 2D. La manera estándar de presentar los espectro de amplitud y fase, es a través de graficas de frecuencia contra amplitud (Fig. 1).

Para lograr un interpretación frecuencia versus amplitud, planteo dos formar de estimar los espectros de Fourier de forma lineal, $A = X(f) $:  

>> **1. Primera técnica:** calcular los coeficientes de Fourier y ángulos complejos de cada una de las trazas sísmicas y después promediarlos entre el número de trazas de la sección sísmica.

>> **2. Segunda técnica:** estimar una traza sísmica equivalente a la sección sísmica y calcular los coeficientes de Fourier y ángulos complejos.

La primera técnica, físicamente es una mejor aproximación a la respuesta espectral de los datos sísmicos bidimensionales.

#### Cálculo de la transformada de Fourier en dos dimensiones

* ####  Amplitud (energía)

In [ ]:
amp = np.abs(np.fft.rfft2(seis))# Magnitud lineal
amp_db = 20*np.log10(np.abs(np.fft.rfft2(seis))) #Magnitud en decibeles 

La función `np.fft.rfftfreq` proporciona el intervalo de frecuencia en Hertz de los datos sísmicos.

In [ ]:
frec = np.fft.rfftfreq(len(seis.T), d = dt)

In [ ]:
print('Tamaño de la matriz Transformada de Fourier = ', amp.T.shape)

In [ ]:
frecmax_idx = amp.T.shape[0]
frec_nyq = 1/(2*0.004)

In [ ]:
print('La frecuencia índice máxima = ', frecmax_idx)
print('La frecuencia de Nyquist = ', frec_nyq, 'Hz')

* #### Fase (ángulo versus frecuencia)

In [ ]:
fase = np.angle((np.fft.rfft2(seis)), deg=True)

In [ ]:
plt.figure(figsize=[14,6])

pot_db = plt.imshow(amp_db.T, aspect='auto', cmap = 'Spectral_r', vmin=65, vmax=130)
cbar= plt.colorbar(pot_db, aspect=20, shrink=1)
cbar.set_label('Amplitud [dB]')
_ = plt.yticks(np.linspace(0,376,12), [0,10,20,30,40,50,60,70,80,90,100,120])
_ = plt.xlabel('No. Traza', fontsize = 10, weight = 'semibold')
_ = plt.ylabel('Frecuencia [Hz]', fontsize = 10, weight = 'semibold')
_ = plt.title('Espectro de amplitud', fontsize=12, weight = 'semibold')
_ = plt.grid(True,alpha=0.6,linestyle=':')

<font size=2 color=black>**Figura 3.** Espectro de amplitud en escala de decibeles. </font>

In [ ]:
plt.figure(figsize=[14,6])

pot_norm = plt.imshow(amp.T, aspect='auto', cmap = 'Spectral_r', vmin=1000, vmax=850000)
cbar= plt.colorbar(pot_norm, aspect=20, shrink=1)
cbar.set_label('Amplitud')
_ = plt.yticks(np.linspace(0,376,12), [0,10,20,30,40,50,60,70,80,90,100,120])
_ = plt.xlabel('No. Traza', fontsize = 10, weight = 'semibold')
_ = plt.ylabel('Frecuencia [Hz]', fontsize = 10, weight = 'semibold')
_ = plt.title('Espectro de amplitud', fontsize=12, weight = 'semibold')
_ = plt.grid(True,alpha=0.6,linestyle=':')

<font size=2 color=black>**Figura 4.** Espectro de amplitud en escala lineal. </font>

In [ ]:
plt.figure(figsize=[14,6])

fase_rad = plt.imshow(fase.T, aspect='auto', cmap = 'brg')
cbar= plt.colorbar(fase_rad, aspect=20, shrink=1)
cbar.set_label('Ángulo [radianes]')
_ = plt.yticks(np.linspace(0,376,12), [0,10,20,30,40,50,60,70,80,90,100,120])
_ = plt.xlabel('No. Traza', fontsize = 10, weight = 'semibold')
_ = plt.ylabel('Frecuencia [Hz]', fontsize = 10, weight = 'semibold')
_ = plt.title('Espectro de fase', fontsize=12, weight = 'semibold')
_ = plt.grid(True,alpha=0.6,linestyle=':')

<font size=2 color=black>**Figura 5.** Espectro de fase en escala de radianes. </font>

### Primera técnica

Al graficar la amplitud versus frecuencia, se obtiene la gráfica espectral para cada una de las 601 trazas sísmica de la sección (Fig. 6); por lo tanto, calculamos la respuesta espectral promedio. Para ello, de la matriz **amp$_{(376,601)}$**, sumamos los renglones y dividimos por el número de trazas sísmicas, obtiendo vector columna **pot$_{(376)}$**. Para esto, ocupamos la función [numpy.matrix.sum](https://docs.scipy.org/doc/numpy-1.15.1/reference/generated/numpy.matrix.sum.html), con el parámetro `axis=0`, en las variables `amp` y `amp_db`.

En la Fig. 7 se muestra el resultado de la respuesta espectral promedio. En la Figs. 8 y 9 se muestra el espectro de amplitud en escala decibel y lineal. En la Fig. 10 se muestra el espectro promedio en escala decibel y lineal en la misma imagen.

* ####  Amplitud (energía)

In [ ]:
plt.figure(figsize=(9,5))

_ = plt.plot(amp_db.T)
_ = plt.ylabel('Amplitud [dB]', fontsize = 10, weight = 'semibold')
_ = plt.xlabel('Frecuencia [Hz]', fontsize = 10, weight = 'semibold')
_ = plt.grid(True,alpha=0.6,linestyle=':')

<font size=2 color=black>**Figura 6.** Espectro de amplitud de las trazas sísmicas de la sección sísmica. </font>

In [ ]:
pot = signal.medfilt(amp.sum(axis=0)/len(seis),11)
pot_db = signal.medfilt(amp_db.sum(axis=0)/len(seis),11)

In [ ]:
plt.figure(figsize=(9,5))

_ = plt.plot(frec, amp_db.T, alpha = 0.017)
_ = plt.plot(frec, pot_db, lw = 2, color = 'b')

_ = plt.ylabel('Amplitud', fontsize = 10, weight = 'semibold')
_ = plt.xlabel('Frecuencia [Hz]', fontsize = 10, weight = 'semibold')
_ = plt.title('Espectro de amplitud [dB]', fontsize=12, weight = 'semibold')
_ = plt.xlim(0,125)
_ = plt.ylim(35,150)
_ = plt.grid(True,alpha=0.6,linestyle=':')

<font size=2 color=black>**Figura 7.** Respuesta espectral promedio. </font>

In [ ]:
plt.figure(figsize=(9,5))


_ = plt.bar(frec, pot_db)
_ = plt.ylabel('Amplitud', fontsize = 10, weight = 'semibold')
_ = plt.xlabel('Frecuencia [Hz]', fontsize = 10, weight = 'semibold')
_ = plt.title('Espectro de amplitud [dB]', fontsize=12, weight = 'semibold')
_ = plt.xlim(0,125)
_ = plt.ylim(65,115)
_ = plt.grid(True,alpha=0.6,linestyle=':')

<font size=2 color=black>**Figura 8.** Espectro de amplitud promedio en escala decibel. </font>

In [ ]:
plt.figure(figsize=(9,5))

_ = plt.bar(frec, pot)
_ = plt.ylabel('Amplitud', fontsize = 10, weight = 'semibold')
_ = plt.xlabel('Frecuencia [Hz]', fontsize = 10, weight = 'semibold')
_ = plt.title('Espectro de amplitud', fontsize=12, weight = 'semibold')
_ = plt.xlim(0,120)
_ = plt.grid(True,alpha=0.6,linestyle=':')

<font size=2 color=black>**Figura 9.** Espectro de amplitud promedio en escala lineal. </font>

In [ ]:
pot_nor = ((pot-min(pot))/(max(pot)-min(pot))) #Espectro de escala lineal normalizado [0,1]

In [ ]:
fig, ax1 = plt.subplots(figsize=(9,5))

color = 'tab:red'
ax1.set_xlabel('Frecuencia [Hz]', fontsize = 10, weight = 'semibold')
ax1.set_ylabel('Amplitud [1]', color = color, fontsize = 10, weight = 'semibold')
ax1.plot(frec, pot_nor, color = color, lw = 2)
ax1.tick_params(axis='y', labelcolor=color)

ax2 = ax1.twinx()

color = 'tab:blue'
ax2.set_ylabel('Amplitud dB [dB]', fontsize = 10, color = color, weight = 'semibold')
ax2.plot(frec, pot_db, lw = 2)
ax2.tick_params(axis='y', labelcolor=color)

plt.title('Espectro de amplitud', fontsize=12, weight = 'semibold')
plt.xlim(0,125)

plt.show()

<font size=2 color=black>**Figura 10.** Espectro de amplitud promedio en escala decibel y normalizado. </font>

* #### Fase (ángulo versus frecuencia)

La Fig. 11 muestra de fondo el espectro de fase de las 601 trazas sísmicas de la sección, así como el espectro promedio de fase en color azul.

In [ ]:
fase_prom = fase.sum(axis=0)/len(seis)

In [ ]:
plt.figure(figsize=(9,5))

fase = np.angle((np.fft.rfft2(seis)), deg=True)

_ = plt.plot(frec, fase.T, alpha = 0.008)
_ = plt.plot(frec, fase_prom, lw = 1, color = 'b')
_ = plt.ylabel('Amplitud [dB]', fontsize = 10, weight = 'semibold')
_ = plt.xlabel('Frecuencia [Hz]', fontsize = 10, weight = 'semibold')
_ = plt.ylim(-200,200)
_ = plt.title('Espectro de fase', fontsize=12, weight = 'semibold')
_ = plt.grid(True,alpha=0.6,linestyle=':')

<font size=2 color=black>**Figura 11.** Espectro de fase promedio y de cada una de las trazas sísmicas de la sección en grados sexagesimales. </font>

### Segunda técnica

Se estima una traza equivalente de la sección sísmica. Con la función [numpy.matrix.sum](https://docs.scipy.org/doc/numpy-1.15.1/reference/generated/numpy.matrix.sum.html), obtenemos la traza promedio. La Fig. 12 muestra la traza sísmica equivalente.

In [ ]:
traza_eq = seis.sum(axis=0)/len(seis)
x = np.arange(0,751,1)

In [ ]:
plt.figure(figsize=(16,3))
_ = plt.plot(traza_eq, color = 'k')
_ = plt.xticks(np.linspace(0,751,7), [0.0,0.5,1.0,1.5,2.0,2.5,3.0])
_ = plt.fill_between(x, traza_eq, 0, traza_eq < 0, color = 'b')
_ = plt.fill_between(x, traza_eq, 0, traza_eq > 0, color = 'r')
_ = plt.ylabel('Amplitud', fontsize = 10, weight = 'semibold')
_ = plt.xlabel('Tiempo [s]', fontsize = 10, weight = 'semibold')
_ = plt.title('Traza sísmica equivalente', fontsize=12, weight = 'semibold')
_ = plt.xlim(0,751)
_ = plt.grid(True,alpha=0.6,linestyle=':')

<font size=2 color=black>**Figura 12.** Traza sísmica equivalente. </font>

* ####  Amplitud (energía)

La Fig. 13 muestra los espectro de amplitud en escala decibel y lineal de la traza sísmica equivalente:

In [ ]:
amp_tr = np.abs(np.fft.rfft(traza_eq))
amp_db_tr = 20*np.log10(np.abs(np.fft.rfft(traza_eq))) 
frec_tr = np.fft.rfftfreq(len(traza_eq), d = dt)

In [ ]:
amp_nor = ((amp_tr-min(amp_tr))/(max(amp_tr)-min(amp_tr))) #Espectro de escala lineal normalizado [0,1]

In [ ]:
fig, ax1 = plt.subplots(figsize=(9,5))

color = 'tab:red'
ax1.set_xlabel('Frecuencia [Hz]', fontsize = 10, weight = 'semibold')
ax1.set_ylabel('Amplitud [dB]', color = color, fontsize = 10, weight = 'semibold')
ax1.bar(frec_tr, signal.medfilt(amp_db_tr,5), color = color, lw = 2, alpha = 0.4)
ax1.tick_params(axis='y', labelcolor=color)

ax2 = ax1.twinx()

color = 'tab:blue'
ax2.set_ylabel('Amplitud [1]', fontsize = 10, color = color, weight = 'semibold')
ax2.bar(frec_tr, signal.medfilt(amp_nor,5), lw = 2, alpha = 0.6)
ax2.tick_params(axis='y', labelcolor=color)

plt.title('Espectro de amplitud', fontsize=12, weight = 'semibold')
plt.xlim(0,125)

plt.show()

<font size=2 color=black>**Figura 13.** Espectro de amplitud en escala decibel y lineal. </font>

* #### Fase (ángulo versus frecuencia)

La Fig. 14 muestra el espectro de fase de la traza sísmica equivalente.

In [ ]:
amp_tra = np.fft.rfft(traza_eq)
fase_traza = np.angle((amp_tra), deg=True)

In [ ]:
plt.figure(figsize=(9,5))

_ = plt.plot(frec_tr,fase_traza)
_ = plt.ylabel('Grados sexagesimales', fontsize = 10, weight = 'semibold')
_ = plt.xlabel('Frecuencia [Hz]', fontsize = 10, weight = 'semibold')
_ = plt.title('Espectro de fase', fontsize=12, weight = 'semibold')
#_ = plt.xlim(0,120)
_ = plt.grid(True,alpha=0.6,linestyle=':')

<font size=2 color=black>**Figura 14.** Espectro de fase en grados sexagesimales. </font>

### Comparación de resultados entre las primera y segunda técnicas 

Las Figs. 15 y 16 muestran las respuestas espectrales de amplitud y fase entre las técnicas empleadas. En mi opinión, concluyo que la primera técnica son la mejor aproximación promedio de las respuestas espectrales. 

In [ ]:
plt.figure(figsize=(9,5))


_ = plt.bar(frec, pot_db, alpha = 0.4, label = 'Tec. no. 1')
_ = plt.bar(frec, signal.medfilt(amp_db_tr,23), alpha = 0.4, label = 'Tec. no. 2')
_ = plt.ylabel('Amplitud [dB]', fontsize = 10, weight = 'semibold')
_ = plt.xlabel('Frecuencia [Hz]', fontsize = 10, weight = 'semibold')
_ = plt.title('Diferencia entre técnicas', fontsize=12, weight = 'semibold')
_ = plt.legend(fontsize = 11)
_ = plt.xlim(0,125)
_ = plt.ylim(20,115)
_ = plt.grid(True,alpha=0.6,linestyle=':')

<font size=2 color=black>**Figura 15.** Espectro de amplitud de las técnicas 1 y 2, respectivamente.</font>

In [ ]:
plt.figure(figsize=(9,5))

_ = plt.plot(frec, fase_prom, lw = 1, label = 'Sol no. 1')
_ = plt.plot(frec_tr,fase_traza, lw = 1, label = 'Sol no. 2')
_ = plt.ylabel('Grados sexagesimales', fontsize = 10, weight = 'semibold')
_ = plt.xlabel('Frecuencia [Hz]', fontsize = 10, weight = 'semibold')
_ = plt.title('Diferencia entre técnicas', fontsize=12, weight = 'semibold')
_ = plt.legend(fontsize = 11)
_ = plt.ylim(-180, 180)

_ = plt.grid(True,alpha=0.6,linestyle=':')

<font size=2 color=black>**Figura 16.** Espectro de fase de las técnicas 1 y 2, respectivamente.</font>

### Trabajos a futuro

>> 1.- Estimar el espectro de amplitud y fase con base en un promedio ponderado que considere la distribución y contribución de los datos. 

>> 2.- Graficar los espectros de amplitud y fase de la sección sísmica obtenidos con algún software y comparar los resultados.